# Snow Density Calculations

This notebook demonstrates methods for calculating density from common snow pit measurements (hand hardness and grain form) using the "Geldsetzer" method for calculating density. The analysis the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [1]:
# Import Libraries
import os
import sys

import numpy as np
import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form
from snowpyt_mechparams import density


Parse Snowpit Files

In [2]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


Collect relevant data for each pit and convert to form needed to implement Geldsetzer

In [3]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'grain_form_primary': layer.grain_form_primary,
        }

        # Add geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'geldsetzer')
        else:
            layer_dict['geldsetzer_grain_form'] = None

        layer_info.append(layer_dict)

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)


In [4]:
# Dataset Summary
print("=== Dataset Summary ===")
print(f"Total number of snowpits: {len(all_pits)}")
print(f"Total number of layers: {len(layer_df)}")

# Calculate statistics directly from layer_df
layers_with_data_count = len(layer_df[layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()])
print(f"Layers with both hand hardness and grain form data: {layers_with_data_count}, "
      f"{layers_with_data_count/len(layer_df)*100:.1f}%")

geldsetzer_layers_count = len(layer_df[layer_df['geldsetzer_grain_form'].notna()])
print(f"Layers with Geldsetzer grain forms: {geldsetzer_layers_count}, "
      f"{geldsetzer_layers_count/len(layer_df)*100:.1f}%")


=== Dataset Summary ===
Total number of snowpits: 50278
Total number of layers: 371429
Layers with both hand hardness and grain form data: 279331, 75.2%
Layers with Geldsetzer grain forms: 221734, 59.7%


Implement Geldsetzer Method

In [5]:
# Calculate density using Geldsetzer method for each layer
def calculate_layer_density(row):
    try:
        # Only calculate if we have a mapped grain form
        if pd.isna(row['geldsetzer_grain_form']):
            return pd.Series([np.nan, np.nan])

        # The density function returns a ufloat object with nominal value and uncertainty
        density_ufloat = density.calculate_density(
            method='geldsetzer',
            hand_hardness=row['hand_hardness'],
            grain_form=row['geldsetzer_grain_form']
        )

        # Extract nominal value and standard deviation from ufloat object
        density_val = density_ufloat.nominal_value  # or density_ufloat.n
        density_unc = density_ufloat.std_dev        # or density_ufloat.s

        return pd.Series([density_val, density_unc])
    except Exception as e:
        print(f"Error calculating density for layer {row['pit_id']}: {e}")
        return pd.Series([np.nan, np.nan])

# Add density and uncertainty columns to the dataframe using apply method
# Calculate density for all rows - the function handles missing data internally
layer_df[['density', 'density_uncertainty']] = layer_df.apply(calculate_layer_density, axis=1)

# Display results summary - filter for successfully calculated densities
successful_layers = layer_df[(layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()) & 
                            layer_df['density'].notna()]
layers_with_data_count = len(layer_df[layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()])
print("=== Density Calculation Results ===")
print(f"Layers with data and calculated density: {len(successful_layers)}")
print(f"Layers with data where density could not be calculated: {layers_with_data_count - len(successful_layers)}")
print(f"Success rate of layers with data: {len(successful_layers)/layers_with_data_count*100:.1f}%")


=== Density Calculation Results ===
Layers with data and calculated density: 200676
Layers with data where density could not be calculated: 78655
Success rate of layers with data: 71.8%


In [ ]:
layer_df.to_csv('geldsetzer_density_results.csv', index=False)


In [7]:
# Detailed uncertainty analysis by grain form
print("=== Uncertainty Analysis by Grain Form ===")
successful_layers = layer_df.dropna(subset=['density', 'density_uncertainty'])

# Group by grain form and calculate statistics
grain_form_stats = successful_layers.groupby('geldsetzer_grain_form').agg({
    'density': ['count', 'mean', 'std'],
    'density_uncertainty': ['mean', 'std']
}).round(1)

# Flatten column names for easier reading
grain_form_stats.columns = [f'{col[1]}_{col[0]}' if col[1] != '' else col[0] 
                           for col in grain_form_stats.columns]

# Rename columns for clarity
grain_form_stats.columns = ['count', 'mean_density', 'density_std', 'mean_uncertainty', 'uncertainty_std']

print(f"{'Grain Form':<12} {'Count':<8} {'Mean Density':<12} {'Mean Uncert.':<12} {'Rel. Uncert.(%)':<15}")
print("-" * 70)

for grain_form, row in grain_form_stats.iterrows():
    relative_uncert = (row['mean_uncertainty'] / row['mean_density']) * 100
    print(f"{grain_form:<12} {row['count']:<8.0f} {row['mean_density']:<12.1f} "
          f"{row['mean_uncertainty']:<12.1f} {relative_uncert:<15.1f}")

print(f"\nNote: Uncertainty values represent the standard error from the Geldsetzer regression models")


=== Uncertainty Analysis by Grain Form ===
Grain Form   Count    Mean Density Mean Uncert. Rel. Uncert.(%)
----------------------------------------------------------------------
DF           38824    135.4        30.0         22.2           
DH           11325    229.8        41.0         17.8           
FC           72974    216.6        43.0         19.9           
PP           17223    89.2         27.0         30.3           
PPgp         2466     151.3        42.0         27.8           
RG           57864    226.2        46.0         20.3           

Note: Uncertainty values represent the standard error from the Geldsetzer regression models


## Uncertainty Analysis Summary

The uncertainty analysis above provides several key insights:

### Uncertainty Sources
- **Standard Error (SE)**: The uncertainty values represent the standard errors from the Geldsetzer regression models, as reported in Table 3 of Geldsetzer & Jamieson (2000)
- **Grain Form Dependence**: Different grain forms have different inherent uncertainties based on how well the regression models fit the original data

### Key Metrics
- **Absolute Uncertainty**: Reported in kg/m³, representing the expected error in density estimates
- **Relative Uncertainty**: Expressed as a percentage of the density value, showing which grain forms have proportionally higher uncertainty
- **Grain Form Analysis**: Shows how uncertainty varies across different snow grain types, helping identify which estimates are most/least reliable

### Interpretation
- Lower relative uncertainty indicates more reliable density estimates for that grain form
- The uncertainty represents the expected variability in density estimates for given hand hardness and grain form combinations
- These uncertainties should be considered when using density estimates in avalanche hazard assessment or snow mechanics calculations
